# Pickle 模块测试套件 - 稳定性与正确性分析


In [42]:
import pickle
import hashlib
import sys
import platform
import datetime
import os


In [ ]:
def get_pickle_hash(obj):
    try:
        data = pickle.dumps(obj)
        return hashlib.sha256(data).hexdigest()
    except Exception as e:
        return f"ERROR: {e}"


## 功能性测试（Functional Testing）：基础类型的序列化


In [ ]:
test_data = {
    "整数": 42,
    "浮点数": 3.14159,
    "布尔值": True,
    "字符串": "ChatGPT",
    "列表": [1, 2, 3],
    "字典": {"a": 1, "b": 2},
    "元组": (4, 5, 6),
    "集合": {7, 8, 9},
    "嵌套结构": {"x": [1, {"y": (3, 4)}]},
    "空值": None
}

results = {}
for name, obj in test_data.items():
    try:
        serialized = pickle.dumps(obj)
        deserialized = pickle.loads(serialized)
        passed = obj == deserialized
    except Exception as e:
        passed = False
        deserialized = str(e)
    results[name] = {
        "原始对象": obj,
        "反序列化结果": deserialized,
        "是否一致": passed
    }

results


{'整数': {'原始对象': 42, '反序列化结果': "name 'pickle' is not defined", '是否一致': False},
 '浮点数': {'原始对象': 3.14159,
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '布尔值': {'原始对象': True,
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '字符串': {'原始对象': 'ChatGPT',
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '列表': {'原始对象': [1, 2, 3],
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '字典': {'原始对象': {'a': 1, 'b': 2},
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '元组': {'原始对象': (4, 5, 6),
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '集合': {'原始对象': {7, 8, 9},
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '嵌套结构': {'原始对象': {'x': [1, {'y': (3, 4)}]},
  '反序列化结果': "name 'pickle' is not defined",
  '是否一致': False},
 '空值': {'原始对象': None, '反序列化结果': "name 'pickle' is not defined", '是否一致': False}}

## 稳定性测试：相同对象的 Pickle 哈希值是否一致


In [ ]:
sample_object = {"test": [1, 2, 3], "time": datetime.datetime(2020, 1, 1)}

hash1 = get_pickle_hash(sample_object)
hash2 = get_pickle_hash(sample_object)
hash3 = get_pickle_hash(sample_object)

print("第一次哈希:", hash1)
print("第二次哈希:", hash2)
print("第三次哈希:", hash3)
print("一致性:", hash1 == hash2 == hash3)


第一次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
第二次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
第三次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
一致性: True


## 模糊测试示例：随机生成复杂对象并检查序列化稳定性


In [ ]:
import random

def generate_random_nested_structure(depth=3):
    if depth == 0:
        return random.choice([42, 3.14, True, "text", None])
    structure_type = random.choice(["list", "dict", "tuple"])
    if structure_type == "list":
        return [generate_random_nested_structure(depth - 1) for _ in range(2)]
    elif structure_type == "dict":
        return {str(i): generate_random_nested_structure(depth - 1) for i in range(2)}
    else:
        return tuple(generate_random_nested_structure(depth - 1) for _ in range(2))

fuzz_object = generate_random_nested_structure()
hash_before = get_pickle_hash(fuzz_object)
hash_after = get_pickle_hash(fuzz_object)

print("生成的结构:", fuzz_object)
print("哈希一致性:", hash_before == hash_after)


生成的结构: ({'0': [None, 42], '1': {'0': 3.14, '1': True}}, [{'0': 42, '1': 3.14}, ('text', 3.14)])
哈希一致性: True


## 反序列化安全与一致性测试

In [43]:
print("=== 传输稳定性测试 ===")
try:
    # 序列化
    pickled_data = pickle.dumps(security_test_data, protocol=pickle.HIGHEST_PROTOCOL)
    original_hash = hashlib.sha256(pickled_data).hexdigest()

    # 压缩和编码
    compressed = gzip.compress(pickled_data)
    encoded = base64.b64encode(compressed)

    # 解码和解压
    decoded = base64.b64decode(encoded)
    decompressed = gzip.decompress(decoded)

    # 反序列化
    restored_object = pickle.loads(decompressed)
    restored_hash = hashlib.sha256(decompressed).hexdigest()

    # 验证
    assert restored_object == security_test_data
    assert restored_hash == original_hash
    print("✅ 传输稳定性测试通过")
except Exception as e:
    print(f"❌ 传输稳定性测试失败: {e}")

=== 传输稳定性测试 ===
✅ 传输稳定性测试通过
